In [2]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path

# ─── Configuration ────────────────────────────────────────────────────────────
neuron_idx = 37827
n_images = 118
n_repeats = 50
k_shot = 50
n_tasks = 200
n_bootstrap = 5
n_pg = 5

neural_data_path = '/home/maria/LuckyMouse4/data/hybrid_neural_responses.npy'
vit_embedding_path = '/home/maria/LuckyMouse4/data/google_vit-base-patch16-224_embeddings_logits.pkl'
report_dir = Path("/home/maria/LuckyMouse4/reports/fewshot_pga_single/")
report_dir.mkdir(parents=True, exist_ok=True)

# ─── Load Data ────────────────────────────────────────────────────────────────
print("Loading data...")
neural_data = np.load(neural_data_path)
with open(vit_embedding_path, 'rb') as f:
    embeddings_raw = pickle.load(f)
image_embeddings = embeddings_raw['natural_scenes']
D = image_embeddings.shape[1]

# ─── Few-Shot Weight Extraction ──────────────────────────────────────────────
def compute_fewshot_weights(neuron_idx, k_shot=15, n_tasks=200):
    neuron_response = neural_data[neuron_idx]
    stimulus_responses = neuron_response.reshape(n_images, n_repeats)
    weight_vectors = []

    for _ in range(n_tasks):
        image_idxs = np.random.choice(n_images, size=k_shot, replace=False)
        X = np.vstack([image_embeddings[i] for i in image_idxs for _ in range(n_repeats)])
        y = np.hstack([stimulus_responses[i] for i in image_idxs])
        X = StandardScaler().fit_transform(X)

        try:
            clf = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=500)
            clf.fit(X, y)
            w = clf.coef_[0]
            w /= np.linalg.norm(w)
            weight_vectors.append(w)
        except Exception:
            continue

    return np.stack(weight_vectors) if weight_vectors else None

# ─── Analysis Helpers ────────────────────────────────────────────────────────
def pca_explained_variance(weights):
    pca = PCA()
    pca.fit(weights)
    return pca.explained_variance_ratio_

def mean_angle(weights):
    cs = cosine_similarity(weights)
    angles = np.degrees(np.arccos(np.clip(cs[np.triu_indices_from(cs,1)], -1, 1)))
    return angles.mean(), angles

def bootstrap_overlap(weights, n_bootstrap=5, n_pg=5):
    overlaps = []
    for _ in range(n_bootstrap):
        idx = np.random.permutation(len(weights))
        split1, split2 = weights[idx[:len(weights)//2]], weights[idx[len(weights)//2:]]
        pca1, pca2 = PCA(n_components=n_pg), PCA(n_components=n_pg)
        pca1.fit(split1); pca2.fit(split2)
        V1, V2 = pca1.components_.T, pca2.components_.T
        overlaps.append(np.trace(V1.T @ V2) / n_pg)
    return np.mean(overlaps)

# ─── Main Procedure ──────────────────────────────────────────────────────────
print(f"Computing few-shot weight vectors for neuron {neuron_idx}...")
weights = compute_fewshot_weights(neuron_idx, k_shot, n_tasks)

if weights is None:
    raise RuntimeError("No valid logistic regression fits. Try increasing tasks or data size.")

# Diagnostics
var_real = pca_explained_variance(weights)
mean_ang, angles = mean_angle(weights)
overlap = bootstrap_overlap(weights, n_bootstrap, n_pg)

# Random baseline
rand_w = np.random.randn(*weights.shape)
rand_w /= np.linalg.norm(rand_w, axis=1, keepdims=True)
var_rand = pca_explained_variance(rand_w)
mean_rand, angles_rand = mean_angle(rand_w)

# PG1 projections
pca = PCA(n_components=1)
proj = pca.fit(weights).transform(weights).flatten()

# ─── Plot results ────────────────────────────────────────────────────────────
fig, axs = plt.subplots(2, 2, figsize=(10, 8))
axs[0,0].plot(var_real[:50], label='Real')
axs[0,0].plot(var_rand[:50], label='Random', linestyle='--')
axs[0,0].set_title("Explained Variance Spectrum")
axs[0,0].legend()

axs[0,1].hist(angles, bins=40, alpha=0.7, label='Real')
axs[0,1].hist(angles_rand, bins=40, alpha=0.7, label='Random')
axs[0,1].set_title(f"Pairwise Angles (mean={mean_ang:.1f}°)")
axs[0,1].legend()

axs[1,0].hist(proj, bins=40)
axs[1,0].set_title("Projections on PG1")

axs[1,1].bar(['Real vs Real'], [overlap])
axs[1,1].set_ylim(0, 1)
axs[1,1].set_title("Bootstrap Subspace Overlap")

fig.suptitle(f"Neuron {neuron_idx} Few-Shot PGA Analysis", fontsize=14)
plt.tight_layout()
plt.savefig(report_dir / f"neuron_{neuron_idx}_fewshot_pga.pdf")
plt.close()

print(f"Report saved to {report_dir}/neuron_{neuron_idx}_fewshot_pga.pdf")
print(f"Mean pairwise angle: {mean_ang:.2f}° (random baseline: {mean_rand:.2f}°)")
print(f"Bootstrap overlap score: {overlap:.3f}")
print("Top 5 explained variances:", var_real[:5])


Loading data...
Computing few-shot weight vectors for neuron 37827...
Report saved to /home/maria/LuckyMouse4/reports/fewshot_pga_single/neuron_37827_fewshot_pga.pdf
Mean pairwise angle: 73.66° (random baseline: 90.00°)
Bootstrap overlap score: 0.012
Top 5 explained variances: [0.03814072 0.03598287 0.03439965 0.03404619 0.03388676]


In [3]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path

# ─── Config ──────────────────────────────────────────────────────────────────
neuron_idx = 37827
k_values = [15, 30, 50, 80]
n_images = 118
n_repeats = 50
n_tasks = 200
n_bootstrap = 5
n_pg = 5

neural_data_path = '/home/maria/LuckyMouse4/data/hybrid_neural_responses.npy'
vit_embedding_path = '/home/maria/LuckyMouse4/data/google_vit-base-patch16-224_embeddings_logits.pkl'
report_dir = Path("/home/maria/LuckyMouse4/reports/fewshot_pga_single/")
report_dir.mkdir(parents=True, exist_ok=True)

# ─── Load Data ───────────────────────────────────────────────────────────────
print("Loading data...")
neural_data = np.load(neural_data_path)
with open(vit_embedding_path, 'rb') as f:
    embeddings_raw = pickle.load(f)
image_embeddings = embeddings_raw['natural_scenes']
D = image_embeddings.shape[1]

# ─── Few-Shot Weight Extraction ──────────────────────────────────────────────
def compute_fewshot_weights(neuron_idx, k_shot=15, n_tasks=200):
    neuron_response = neural_data[neuron_idx]
    stimulus_responses = neuron_response.reshape(n_images, n_repeats)
    weight_vectors = []

    for _ in range(n_tasks):
        image_idxs = np.random.choice(n_images, size=k_shot, replace=False)
        X = np.vstack([image_embeddings[i] for i in image_idxs for _ in range(n_repeats)])
        y = np.hstack([stimulus_responses[i] for i in image_idxs])
        X = StandardScaler().fit_transform(X)

        try:
            clf = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=500)
            clf.fit(X, y)
            w = clf.coef_[0]
            w /= np.linalg.norm(w)
            weight_vectors.append(w)
        except Exception:
            continue

    return np.stack(weight_vectors) if weight_vectors else None

# ─── Analysis Helpers ────────────────────────────────────────────────────────
def pca_explained_variance(weights):
    pca = PCA()
    pca.fit(weights)
    return pca.explained_variance_ratio_

def mean_angle(weights):
    cs = cosine_similarity(weights)
    angles = np.degrees(np.arccos(np.clip(cs[np.triu_indices_from(cs,1)], -1, 1)))
    return angles.mean(), angles

def grassmann_overlap(weights, n_bootstrap=5, n_pg=5):
    overlaps = []
    for _ in range(n_bootstrap):
        idx = np.random.permutation(len(weights))
        split1, split2 = weights[idx[:len(weights)//2]], weights[idx[len(weights)//2:]]
        pca1, pca2 = PCA(n_components=n_pg), PCA(n_components=n_pg)
        pca1.fit(split1); pca2.fit(split2)
        U1, _ = np.linalg.qr(pca1.components_.T)
        U2, _ = np.linalg.qr(pca2.components_.T)
        # Principal angles
        _, s, _ = np.linalg.svd(U1.T @ U2)
        overlap_score = (s**2).mean()  # mean cos²(θ)
        overlaps.append(overlap_score)
    return np.mean(overlaps)

# ─── Run for Multiple k-shot Values ──────────────────────────────────────────
results = {}

for k in k_values:
    print(f"\n--- Running analysis for k={k} ---")
    weights = compute_fewshot_weights(neuron_idx, k_shot=k, n_tasks=n_tasks)
    if weights is None:
        print("No valid weights, skipping.")
        continue

    # Real data stats
    var_real = pca_explained_variance(weights)
    mean_ang, angles = mean_angle(weights)
    overlap = grassmann_overlap(weights, n_bootstrap, n_pg)

    # Random baseline
    rand_w = np.random.randn(*weights.shape)
    rand_w /= np.linalg.norm(rand_w, axis=1, keepdims=True)
    var_rand = pca_explained_variance(rand_w)
    mean_rand, angles_rand = mean_angle(rand_w)

    # PG1 projections
    pca = PCA(n_components=1)
    proj = pca.fit(weights).transform(weights).flatten()

    results[k] = {
        "weights": weights,
        "var_real": var_real,
        "var_rand": var_rand,
        "mean_angle": mean_ang,
        "mean_angle_rand": mean_rand,
        "angles": angles,
        "angles_rand": angles_rand,
        "proj": proj,
        "overlap": overlap
    }

# ─── Generate Combined PDF ───────────────────────────────────────────────────
fig, axs = plt.subplots(len(results), 4, figsize=(18, 4 * len(results)))

for i, k in enumerate(results):
    res = results[k]

    axs[i,0].plot(res["var_real"][:50], label='Real')
    axs[i,0].plot(res["var_rand"][:50], label='Random', linestyle='--')
    axs[i,0].set_title(f"k={k} - Explained Variance")
    axs[i,0].legend()

    axs[i,1].hist(res["angles"], bins=40, alpha=0.7, label='Real')
    axs[i,1].hist(res["angles_rand"], bins=40, alpha=0.7, label='Random')
    axs[i,1].set_title(f"Pairwise Angles (mean={res['mean_angle']:.1f}°)")
    axs[i,1].legend()

    axs[i,2].hist(res["proj"], bins=40)
    axs[i,2].set_title("Projections on PG1")

    axs[i,3].bar(['Grassmann overlap'], [res["overlap"]])
    axs[i,3].set_ylim(0, 1)
    axs[i,3].set_title(f"Subspace Overlap = {res['overlap']:.2f}")

plt.suptitle(f"Neuron {neuron_idx} Few-Shot PGA Geometry across k-values", fontsize=16)
plt.tight_layout()
plt.savefig(report_dir / f"neuron_{neuron_idx}_fewshot_pga_multik.pdf")
plt.close()

print(f"\nReport saved to {report_dir}/neuron_{neuron_idx}_fewshot_pga_multik.pdf")


Loading data...

--- Running analysis for k=15 ---

--- Running analysis for k=30 ---

--- Running analysis for k=50 ---

--- Running analysis for k=80 ---

Report saved to /home/maria/LuckyMouse4/reports/fewshot_pga_single/neuron_37827_fewshot_pga_multik.pdf
